# Malicious Named Pipe
Detects the creation of a named pipe used by known APT malware

## Rule Content
```
- title: Malicious Named Pipe
  id: fe3ac066-98bb-432a-b1e7-a5229cb39d4a
  status: experimental
  description: Detects the creation of a named pipe used by known APT malware
  references:
  - Various sources
  date: 2017/11/06
  author: Florian Roth
  logsource:
    product: windows
    service: sysmon
    definition: Note that you have to configure logging for PipeEvents in Symson config
    category: null
  detection:
    selection:
      EventID:
      - 17
      - 18
      PipeName:
      - \isapi_http
      - \isapi_dg
      - \isapi_dg2
      - \sdlrpc
      - \ahexec
      - \winsession
      - \lsassw
      - \46a676ab7f179e511e30dd2dc41bd388
      - \9f81f59bc58452127884ce513865ed20
      - \e710f28d59aa529d6792ca6ff0ca1b34
      - \rpchlp_3
      - \NamePipe_MoreWindows
      - \pcheap_reuse
      - \msagent_*
    condition: selection
  tags:
  - attack.defense_evasion
  - attack.privilege_escalation
  - attack.t1055
  falsepositives:
  - Unkown
  level: critical

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-sysmon-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(event_id:("17" OR "18") AND pipe_name.keyword:(\\isapi_http OR \\isapi_dg OR \\isapi_dg2 OR \\sdlrpc OR \\ahexec OR \\winsession OR \\lsassw OR \\46a676ab7f179e511e30dd2dc41bd388 OR \\9f81f59bc58452127884ce513865ed20 OR \\e710f28d59aa529d6792ca6ff0ca1b34 OR \\rpchlp_3 OR \\NamePipe_MoreWindows OR \\pcheap_reuse OR \\msagent_*))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()